#### 전결합층 모델 생성
- Full-Connected Layer로 FC라고 명명
- 특징
    * 모든 입력 피쳐/데이터와 가중치(w) 곱셉 후 합계 즉, 가중합 계산 + 절편
    * 가중합 + 절편
- 필요 요소
    * 가중치와 절편 텐서 -> 학습에 의해 업데이트 진행이 되어야 함 -> 변경이 가능해야한다 tf.Variable()
    * 가중치 개수 : 입력 피처 수
    * 절편   개수 : 1개
    * 출력   개수 : 층에 존재 하는 퍼셉트론 개수

[1] 모듈 로딩

In [47]:
import tensorflow as tf
import numpy as np

[2] 모델 클래스 설계 및 정의

In [48]:
### 데  이   터 : 1개 피쳐 , 1개 타겟 
### 학 습 종 류 : 지도/비지도/강화 --> 지도 --> 회귀/분류  : 지도/회귀
### 클래스 기능 : 스칼라 데이터 처리 모델
### 클래스 이름 : SimpleModel
### 클래스 속성 : W, b

class SimpleModel(tf.Module):
    #모델 인스턴스 생성 메서드
    def __init__(self):
        super().__init__()
        self.W = tf.Variable(tf.random.normal((1,)))
        self.b = tf.zeros((1,))

    #학습 메서드
    def __call__(self, input_data):
        return self.W * input_data + self.b

In [49]:
### 데 이 터 셋 : (8,3)
### 피쳐  shape : (8,2)
### 타겟  shape : (8,1)
### 학 습 종 류 : 지도/비지도/강화 --> 지도
### 클래스 목적 : 전결합 처리 모델
### 클래스 이름 : DenseModel
### 클래스 속성 : W, b
### 클래스 기능 : 가중합 + b 반환

class DenseModel(tf.Module):
    #모델 인스턴스 생성 메서드
    # in_nums : 피쳐 수, out_nums : 층의 퍼센트롭 개수(뉴런수)
    def __init__(self, input_nums, out_nums):
        super().__init__()
        self.W = tf.Variable(tf.random.normal((input_nums, out_nums)))
        self.b = tf.Variable(tf.zeros((out_nums)))
    
    #학습 진행 메서드
    def __call__(self, input):
        wbsum = tf.matmul(input, self.W) + self.b

        return wbsum


- 가중합 연산 동작 원리

In [50]:
out_nums = 5
W = tf.Variable(tf.random.normal((2, out_nums)))
b = tf.Variable(tf.zeros((out_nums))) 
x = tf.constant([[2,4],[3,4], [5,6], [5,7], [8,9]], dtype='float32')

print(f'w -> {W.shape}, b -> {b.shape}, x->{x.shape}')
print(f'tf.matmul(x, W) + b = {tf.matmul(x, W) + b}')

w -> (2, 5), b -> (5,), x->(5, 2)
tf.matmul(x, W) + b = [[ 4.992835   3.8174655  7.9467354  9.12069   -1.3494571]
 [ 5.284107   4.3565445  9.037457  11.047821  -1.9935997]
 [ 8.071796   6.8043556 14.101547  17.535295  -3.312471 ]
 [ 9.17437    7.489183  15.54287   18.851902  -3.3277638]
 [12.253331  10.476072  21.697681  27.26651   -5.290777 ]]


- 주의 사항 ! 곱셈 연산시 타입 일치가 필요!!

In [51]:
W = tf.Variable(tf.random.normal((1,)))
b = tf.zeros((1,))
data = tf.constant(2, dtype=tf.float32) # 타입 일치를 위해 형변환 설정

print(f'w --> {W.shape}, b --> {b.shape}, data --> {data.shape}')
print(f'data * W + b = {data * W + b}')

w --> (1,), b --> (1,), data --> ()
data * W + b = [-1.3383757]


[3] 모델 인스턴스 생성 및 학습

In [52]:
### 인스턴스 생성
model = SimpleModel()

In [53]:
### 학습
model( tf.constant(2, dtype='float32') )

<tf.Tensor: shape=(1,), dtype=float32, numpy=array([-3.0190547], dtype=float32)>

- 전결합층 모델 인스턴스 처리

In [54]:
## 모델 인스턴스 생성
fcModel = DenseModel(4, 1)

In [55]:
## 데이터셋
dataDS = tf.constant([[1,2,3,4], [1,4,5,2]], dtype=tf.float32)
dataDS.shape

TensorShape([2, 4])

In [60]:
resultDS = fcModel(dataDS)
print(resultDS)

tf.Tensor(
[[-2.1506042 ]
 [-0.46561563]], shape=(2, 1), dtype=float32)


- 활성화 함수 적용

In [62]:
### 시그모이드 AF : 0.0 ~ 1.0 확률값 변환
tf.nn.sigmoid(resultDS)

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[0.10427477],
       [0.3856545 ]], dtype=float32)>

In [63]:
## 렐루 AF : 0 <= ~ 결과값 반환
tf.nn.relu(resultDS).numpy()

array([[0.],
       [0.]], dtype=float32)

In [64]:
## 리키렐루 AF : 0.00xx <= ~ 결과값 반환
tf.nn.leaky_relu(resultDS).numpy()

array([[-0.43012086],
       [-0.09312313]], dtype=float32)

In [65]:
## 소프트맥스(다중분류에 많이 쓰임) AF : 0.0 <= ~ <= 1.0 확률값 변환
tf.nn.softmax(resultDS).numpy()

array([[1.],
       [1.]], dtype=float32)